In [1]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier

In [2]:
features = dict(np.load("../feature/time_feature.npz"))
labels = dict(np.load("../feature/label.npz"))

feature = None
label = None
for key in features:
    if feature is None:
        feature = features[key]
        label = labels[key]
    else:
        feature = np.concatenate((feature, features[key]))
        label = np.concatenate((label, labels[key]))

In [3]:
records = {
    "ann": np.zeros(10),
    "rf": np.zeros(10),
    "svm": np.zeros(10)
}

In [4]:
for k in range(10):
    clfs = {
        "ann": MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(128), random_state=1),
        "rf": RandomForestClassifier(n_estimators=10, max_depth=None, min_samples_split=2, random_state=0),
        "svm": SVC(gamma='scale', decision_function_shape='ovo')
    }
    #shuffle
    perm = np.random.permutation(feature.shape[0])
    feature = feature[perm]
    label = label[perm]
    
    radio = 0.8

    train_index = slice(0, int(radio * feature.shape[0]))
    valid_index = slice(int(radio * feature.shape[0]), feature.shape[0])

    x_train = feature[train_index]
    y_train = label[train_index]
    x_valid = feature[valid_index]
    y_valid = label[valid_index]
    scaler = StandardScaler()
    x_train_norm = scaler.fit_transform(x_train)
    x_valid_norm = scaler.transform(x_valid)
    
    for key, clf in clfs.items():
        clf.fit(x_train_norm, y_train)
        y_valid_predict = clf.predict(x_valid_norm)
        records[key][k] = np.sum(y_valid_predict.reshape([-1, 1]) == y_valid) / len(y_valid)

D:\Software\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:934: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\Software\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
D:\Software\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
D:\Software\Anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversion

D:\Software\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:934: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\Software\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
D:\Software\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
D:\Software\Anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversion

In [5]:
for key, value in records.items():
    print("%s: %f ± %f" % (key, value.mean(), value.std()))

ann: 0.621474 ± 0.023443
rf: 0.675000 ± 0.010475
svm: 0.526643 ± 0.008911
